## Settings

In [6]:
from torch.utils.data import DataLoader, random_split
from torchvision.transforms import Resize, Compose, ToTensor
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchsummary import summary
from torch import optim
import torch.nn as nn
import torch

from PIL import Image


from DatasetClassifier.dataset.Dataset import CustomImageDataset
from model.BinaryClassifier import BinaryClassifier

import copy, wandb
from sys import float_info
from tqdm import tqdm   


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
generator = torch.Generator().manual_seed(42)

dataset_dir = "BinaryClassifierDataset"

training_settings = {
    "epochs" : 100,
    "batch_size" : 32,
    "learning_rate" : 3e-4,
    "verbose" : False,  
    "logging" : True,
    "save_path" : "DatasetClassifier\\models"
}


# Dataset Transformations
preprocessor = Compose([
    ToTensor(), 
    Resize((640, 640), antialias=True)
])

## Train Functions

In [ ]:
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def loss_batch(loss_func, predictions, target_labels, optimizer):
    
    loss_value = loss_func(predictions, target_labels) 
    pred = predictions.argmax(dim=1, keepdim=True)
    metric_b = pred.eq(target_labels.view_as(pred)).sum().item() # get performance metric
    
    if optimizer is not None:
        optimizer.zero_grad()
        loss_value.backward()
        optimizer.step()

    return loss_value.item(), metric_b


# Compute the loss value & performance metric for the entire dataset (epoch)
def loss_epoch(model, loss_func, dataloader: DataLoader, optimizer):
    
    loss_e = 0.0 
    metric_e = 0.0
    len_data = len(dataloader.dataset)

    # Loop over dataset
    progress_bar = tqdm(dataloader)
    for images, labels in progress_bar:

        images = images.to(device)
        labels = labels.to(device)

        predictions = model(images)
        # Get loss per batch
        loss_b, metric_b = loss_batch(loss_func, predictions, labels, optimizer) 

        if optimizer is not None:
            progress_bar.set_description(f"LR: {get_lr(optimizer)} | Loss: {loss_b:.3f} | Acc: {(metric_b/dataloader.batch_size):.3f}")
            if training_settings["logging"]:    
                wandb.log({"batch_train_accuracy": metric_b, "batch_train_loss": loss_b})
        else:
            progress_bar.set_description(f"Loss: {loss_b:.3f} | Acc: {(metric_b/dataloader.batch_size):.3f}")
            if training_settings["logging"]:    
                wandb.log({"batch_val_accuracy": metric_b, "batch_val_loss": loss_b})

        loss_e += loss_b        

        if metric_b is not None: 
            metric_e += metric_b    
    
    loss = loss_e / float(len_data)  # average loss value
    metric = metric_e / float(len_data) # average metric value
    
    return loss, metric


## Train-Validation LOOP

In [ ]:
if training_settings["logging"]:
    wandb.init(
    project="BinaryClassifier",
    config={
        "learning_rate": training_settings["learning_rate"],
        "architecture": "CNN",
        "dataset": "Discarded RoboTo Images",
        "epochs": training_settings["epochs"],
        }
    )


model = BinaryClassifier(dimensions= (3, 640, 640), num_classes= 2).to(device=device)
dataset = CustomImageDataset(annotations_file= dataset_dir+"\\labels\\labels.csv", 
                                img_dir= dataset_dir+"\\images",
                                transform= preprocessor)

if training_settings["verbose"]:
    summary(model, input_size=(3, 640, 640), device=device.type)
    print(f"Model Output: {model(torch.rand(1, 3, 640, 640))}")

train_dataset, valid_dataset = random_split(dataset, [0.8, 0.2], generator=generator)
train_dataloader = DataLoader(dataset=train_dataset, batch_size=training_settings["batch_size"], shuffle=True)
valid_dataloader = DataLoader(dataset=valid_dataset, batch_size=training_settings["batch_size"], shuffle=False)

# TRAINING
loss_func = nn.NLLLoss(reduction="sum")
optimizer = optim.Adam(model.parameters(), lr=training_settings["learning_rate"])
lr_scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20, verbose=1)
best_loss = float_info.max

progress_bar = tqdm(range(training_settings["epochs"]))
for i, epoch in enumerate(progress_bar):

    model.train()
    train_loss, train_metric = loss_epoch(model, loss_func, train_dataloader, optimizer)
    if training_settings["logging"]:
        wandb.log({"epoch_train_accuracy": train_metric, "epoch_train_loss": train_loss})

    model.eval()
    with torch.no_grad():
        val_loss, val_metric = loss_epoch(model, loss_func, valid_dataloader, None)
        
        if training_settings["logging"]:
            wandb.log({"epoch_val_accuracy": val_metric, "epoch_val_loss": val_loss})
    
    progress_bar.set_description(f"Epoch: {i+1} | LR: {get_lr(optimizer):.3f} | Loss: {train_loss:.3f} | Acc: {train_metric:.3f}")

    # Store best model
    if(val_loss < best_loss):
        best_loss = val_loss
        best_model_wts = copy.deepcopy(model.state_dict())
        
        # store weights into a local file
        torch.save(model.state_dict(), training_settings["save_path"]+"\\best_model.pt")
        if training_settings["verbose"]:
            print("Copied best model weights!")

if training_settings["logging"]:
    wandb.finish()

## Inference

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
generator = torch.Generator().manual_seed(42)

model = BinaryClassifier(dimensions= (3, 640, 640), num_classes= 2)
model.load_state_dict(torch.load("trained_model/best_model.pt", map_location = device))
model = model.to(device)
model.eval()

image_path = "./test_img.jpg"
image = Image.open(image_path)
image = preprocessor(image)
image = image.to(device)

with torch.no_grad():
  output = model(image).argmax(dim=1, keepdim=True)

  print("Allowed" if output.item() else "Not Allowed")
  

Not Allowed
